In [217]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization


In [218]:
data=pd.read_csv("computer_prices_all.csv")

In [219]:
data


,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,...,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
0,Desktop,Samsung,Samsung Forge XDI,2022,Windows,ATX,Intel,Intel i5-11129,3,12,...,2560x1440,90,0,0,750,Wi-Fi 6,5.1,11.00,36,1383.99
1,Laptop,Samsung,Samsung Pro KM8,2022,Windows,Mainstream,Intel,Intel i7-11114,4,12,...,1920x1080,90,56,120,0,Wi-Fi 6,5.3,2.03,12,2274.99
2,Desktop,Lenovo,Lenovo Strix BIE,2024,macOS,SFF,AMD,AMD Ryzen 5 5168,2,8,...,3440x1440,120,0,0,850,Wi-Fi 6,5.0,7.00,24,1879.99
3,Desktop,Dell,Dell Cube AXR,2024,Windows,ATX,AMD,AMD Ryzen 5 7550,2,6,...,3440x1440,120,0,0,650,Wi-Fi 6,5.2,6.00,36,1331.99
4,Laptop,Gigabyte,Gigabyte Pro IX1,2024,Linux,Gaming,AMD,AMD Ryzen 7 6230,5,16,...,2560x1600,90,80,90,0,Wi-Fi 6,5.2,1.50,12,2681.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Laptop,ASUS,ASUS Pro ZWL,2023,Windows,Mainstream,Intel,Intel i7-13721,4,12,...,1920x1080,144,90,180,0,Wi-Fi 6,5.1,1.87,24,1712.99
99996,Laptop,Lenovo,Lenovo Stealth 014,2018,Windows,Ultrabook,AMD,AMD Ryzen 5 5117,2,8,...,2560x1600,90,50,65,0,Wi-Fi 6,5.1,1.37,12,1258.99
99997,Laptop,ASUS,ASUS Zen LKD,2020,Windows,Mainstream,Intel,Intel i5-12677,2,6,...,2560x1600,120,99,180,0,Wi-Fi 6,4.2,1.17,12,1686.99
99998,Laptop,ASUS,ASUS Blade DH6,2020,Windows,Mainstream,AMD,AMD Ryzen 7 4590,4,12,...,2560x1600,120,60,90,0,Wi-Fi 6,5.3,1.70,24,2164.99


In [220]:
data.columns

Index(['device_type', 'brand', 'model', 'release_year', 'os', 'form_factor',
       'cpu_brand', 'cpu_model', 'cpu_tier', 'cpu_cores', 'cpu_threads',
       'cpu_base_ghz', 'cpu_boost_ghz', 'gpu_brand', 'gpu_model', 'gpu_tier',
       'vram_gb', 'ram_gb', 'storage_type', 'storage_gb',
       'storage_drive_count', 'display_type', 'display_size_in', 'resolution',
       'refresh_hz', 'battery_wh', 'charger_watts', 'psu_watts', 'wifi',
       'bluetooth', 'weight_kg', 'warranty_months', 'price'],
      dtype='str')

In [221]:
data.isna().sum()

device_type            0
brand                  0
model                  0
release_year           0
os                     0
form_factor            0
cpu_brand              0
cpu_model              0
cpu_tier               0
cpu_cores              0
cpu_threads            0
cpu_base_ghz           0
cpu_boost_ghz          0
gpu_brand              0
gpu_model              0
gpu_tier               0
vram_gb                0
ram_gb                 0
storage_type           0
storage_gb             0
storage_drive_count    0
display_type           0
display_size_in        0
resolution             0
refresh_hz             0
battery_wh             0
charger_watts          0
psu_watts              0
wifi                   0
bluetooth              0
weight_kg              0
warranty_months        0
price                  0
dtype: int64

In [222]:
data["device_type"]=(data["device_type"]=="Laptop").astype("int8")

In [223]:
data=pd.get_dummies(data,columns=["brand"],dtype="int8")


In [224]:
data["release_year"].value_counts()

release_year
2023    18020
2024    17969
2022    16054
2025    15835
2021    12016
2020     7986
2019     7090
2018     5030
Name: count, dtype: int64

In [225]:
data=pd.get_dummies(data,columns=["release_year"],dtype="int8")    
data

,device_type,model,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,cpu_threads,cpu_base_ghz,...,brand_Razer,brand_Samsung,release_year_2018,release_year_2019,release_year_2020,release_year_2021,release_year_2022,release_year_2023,release_year_2024,release_year_2025
0,0,Samsung Forge XDI,Windows,ATX,Intel,Intel i5-11129,3,12,24,2.8,...,0,1,0,0,0,0,1,0,0,0
1,1,Samsung Pro KM8,Windows,Mainstream,Intel,Intel i7-11114,4,12,24,2.6,...,0,1,0,0,0,0,1,0,0,0
2,0,Lenovo Strix BIE,macOS,SFF,AMD,AMD Ryzen 5 5168,2,8,16,2.6,...,0,0,0,0,0,0,0,0,1,0
3,0,Dell Cube AXR,Windows,ATX,AMD,AMD Ryzen 5 7550,2,6,12,2.6,...,0,0,0,0,0,0,0,0,1,0
4,1,Gigabyte Pro IX1,Linux,Gaming,AMD,AMD Ryzen 7 6230,5,16,32,2.8,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,ASUS Pro ZWL,Windows,Mainstream,Intel,Intel i7-13721,4,12,24,2.6,...,0,0,0,0,0,0,0,1,0,0
99996,1,Lenovo Stealth 014,Windows,Ultrabook,AMD,AMD Ryzen 5 5117,2,8,16,2.2,...,0,0,1,0,0,0,0,0,0,0
99997,1,ASUS Zen LKD,Windows,Mainstream,Intel,Intel i5-12677,2,6,10,2.2,...,0,0,0,0,1,0,0,0,0,0
99998,1,ASUS Blade DH6,Windows,Mainstream,AMD,AMD Ryzen 7 4590,4,12,24,2.6,...,0,0,0,0,1,0,0,0,0,0


In [226]:
data=data.drop(columns=["model"])

In [227]:
data=pd.get_dummies(data,columns=["os"])

In [228]:
data["form_factor"].value_counts()

form_factor
Mainstream     17819
Gaming         16876
ATX            15597
Ultrabook      13236
Micro-ATX       8672
Full-Tower      7110
2-in-1          7049
SFF             5585
Workstation     4864
Mini-ITX        3192
Name: count, dtype: int64

In [229]:
top4 = data["form_factor"].value_counts().index[:4] 
data['form_factor_mod'] = data['form_factor'].apply(lambda x: x if x in top4 else 'Other')
data = pd.get_dummies(data, columns=['form_factor_mod'], prefix='form_factor', dtype='int8')


In [230]:
data["cpu_brand"].value_counts()

cpu_brand
Intel    52774
AMD      35311
Apple    11915
Name: count, dtype: int64

In [231]:
data.columns

Index(['device_type', 'form_factor', 'cpu_brand', 'cpu_model', 'cpu_tier',
       'cpu_cores', 'cpu_threads', 'cpu_base_ghz', 'cpu_boost_ghz',
       'gpu_brand', 'gpu_model', 'gpu_tier', 'vram_gb', 'ram_gb',
       'storage_type', 'storage_gb', 'storage_drive_count', 'display_type',
       'display_size_in', 'resolution', 'refresh_hz', 'battery_wh',
       'charger_watts', 'psu_watts', 'wifi', 'bluetooth', 'weight_kg',
       'warranty_months', 'price', 'brand_ASUS', 'brand_Acer', 'brand_Apple',
       'brand_Dell', 'brand_Gigabyte', 'brand_HP', 'brand_Lenovo', 'brand_MSI',
       'brand_Razer', 'brand_Samsung', 'release_year_2018',
       'release_year_2019', 'release_year_2020', 'release_year_2021',
       'release_year_2022', 'release_year_2023', 'release_year_2024',
       'release_year_2025', 'os_ChromeOS', 'os_Linux', 'os_Windows',
       'os_macOS', 'form_factor_ATX', 'form_factor_Gaming',
       'form_factor_Mainstream', 'form_factor_Other', 'form_factor_Ultrabook'],
      dty

In [232]:
data

,device_type,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,cpu_threads,cpu_base_ghz,cpu_boost_ghz,gpu_brand,...,release_year_2025,os_ChromeOS,os_Linux,os_Windows,os_macOS,form_factor_ATX,form_factor_Gaming,form_factor_Mainstream,form_factor_Other,form_factor_Ultrabook
0,0,ATX,Intel,Intel i5-11129,3,12,24,2.8,3.8,NVIDIA,...,0,False,False,True,False,1,0,0,0,0
1,1,Mainstream,Intel,Intel i7-11114,4,12,24,2.6,3.6,NVIDIA,...,0,False,False,True,False,0,0,1,0,0
2,0,SFF,AMD,AMD Ryzen 5 5168,2,8,16,2.6,3.6,NVIDIA,...,0,False,False,False,True,0,0,0,1,0
3,0,ATX,AMD,AMD Ryzen 5 7550,2,6,12,2.6,3.6,AMD,...,0,False,False,True,False,1,0,0,0,0
4,1,Gaming,AMD,AMD Ryzen 7 6230,5,16,32,2.8,3.9,NVIDIA,...,0,False,True,False,False,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,Mainstream,Intel,Intel i7-13721,4,12,24,2.6,3.7,AMD,...,0,False,False,True,False,0,0,1,0,0
99996,1,Ultrabook,AMD,AMD Ryzen 5 5117,2,8,16,2.2,3.2,AMD,...,0,False,False,True,False,0,0,0,0,1
99997,1,Mainstream,Intel,Intel i5-12677,2,6,10,2.2,3.2,NVIDIA,...,0,False,False,True,False,0,0,1,0,0
99998,1,Mainstream,AMD,AMD Ryzen 7 4590,4,12,24,2.6,3.5,NVIDIA,...,0,False,False,True,False,0,0,1,0,0


In [233]:
data=pd.get_dummies(data,columns=["cpu_brand"], dtype='int8')


In [234]:
top9 =  data["cpu_model"].value_counts()[:9]
data['cpu_model_mod'] = data["cpu_model"].apply(lambda x: x if x in top9 else 'Other')
data=pd.get_dummies(data,columns=['cpu_model_mod'], prefix="cpu_model", dtype='int8')


In [235]:
data

,device_type,form_factor,cpu_model,cpu_tier,cpu_cores,cpu_threads,cpu_base_ghz,cpu_boost_ghz,gpu_brand,gpu_model,...,cpu_model_Apple M1,cpu_model_Apple M1 Max,cpu_model_Apple M1 Pro,cpu_model_Apple M2,cpu_model_Apple M2 Max,cpu_model_Apple M2 Pro,cpu_model_Apple M3,cpu_model_Apple M3 Max,cpu_model_Apple M3 Pro,cpu_model_Other
0,0,ATX,Intel i5-11129,3,12,24,2.8,3.8,NVIDIA,RTX 40 60,...,0,0,0,0,0,0,0,0,0,1
1,1,Mainstream,Intel i7-11114,4,12,24,2.6,3.6,NVIDIA,RTX 40 80,...,0,0,0,0,0,0,0,0,0,1
2,0,SFF,AMD Ryzen 5 5168,2,8,16,2.6,3.6,NVIDIA,RTX 40 50,...,0,0,0,0,0,0,0,0,0,1
3,0,ATX,AMD Ryzen 5 7550,2,6,12,2.6,3.6,AMD,RX 7000 60,...,0,0,0,0,0,0,0,0,0,1
4,1,Gaming,AMD Ryzen 7 6230,5,16,32,2.8,3.9,NVIDIA,RTX 30 80 Ti,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,Mainstream,Intel i7-13721,4,12,24,2.6,3.7,AMD,RX 7000 70,...,0,0,0,0,0,0,0,0,0,1
99996,1,Ultrabook,AMD Ryzen 5 5117,2,8,16,2.2,3.2,AMD,RX 6000 50,...,0,0,0,0,0,0,0,0,0,1
99997,1,Mainstream,Intel i5-12677,2,6,10,2.2,3.2,NVIDIA,RTX 20 60,...,0,0,0,0,0,0,0,0,0,1
99998,1,Mainstream,AMD Ryzen 7 4590,4,12,24,2.6,3.5,NVIDIA,RTX 30 70,...,0,0,0,0,0,0,0,0,0,1


In [236]:
data=pd.get_dummies(data,columns=["cpu_tier"], dtype='int8')

In [237]:
data["cpu_cores"].value_counts()[:14]

cpu_cores
8     24114
6     17723
12    15705
16     9368
4      8655
10     8510
14     6641
18     3858
24     3256
26     1449
20      521
28      200
Name: count, dtype: int64

In [238]:
top3 =  data["cpu_cores"].value_counts()[:3]
data["cpu_cores_mod"] = data["cpu_cores"].apply(lambda x: x if x in top3 else 'Other')
data=pd.get_dummies(data,columns=["cpu_cores_mod"], prefix="cpu_cores", dtype='int8')


In [239]:
data["cpu_threads"].value_counts()

cpu_threads
16    20993
12    16641
24    13264
8      9913
32     7004
20     6893
28     5306
14     3027
36     2907
10     2542
6      2322
48     2118
18     1428
22     1297
52      960
26      917
30      826
4       433
40      364
34      323
46      232
56      127
50      102
38       49
54       12
Name: count, dtype: int64

In [240]:
top14 =  data["cpu_threads"].value_counts()[:14]
data["cpu_threads_mod"] = data["cpu_threads"].apply(lambda x: x if x in top14 else 'Other')
data=pd.get_dummies(data,columns=["cpu_threads_mod"], prefix="cpu_threads", dtype='int8')


In [241]:
data["cpu_base_ghz"].value_counts()

cpu_base_ghz
2.6    22156
2.4    20581
2.8    18139
2.2    12612
3.0    11693
2.0     7749
3.2     5092
3.4     1978
Name: count, dtype: int64

In [242]:
data=pd.get_dummies(data,columns=["cpu_base_ghz"], dtype='int8')

In [243]:
data["cpu_boost_ghz"].value_counts()

cpu_boost_ghz
3.5    10804
3.4    10612
3.6    10350
3.7     9683
3.3     9053
3.2     7874
3.8     7688
3.9     6945
3.1     5632
3.0     4763
4.0     4510
4.1     3540
2.9     2730
4.2     1924
2.8     1584
4.3     1416
4.4      605
4.5      287
Name: count, dtype: int64

In [244]:
top3 =  data["cpu_boost_ghz"].value_counts()[:3]
data["cpu_boost_ghz_mod"] = data["cpu_boost_ghz"].apply(lambda x: x if x in top3 else 'Other')
data=pd.get_dummies(data,columns=["cpu_boost_ghz_mod"], prefix="cpu_boost_ghz", dtype='int8')


In [245]:
data["gpu_brand"].value_counts()

gpu_brand
NVIDIA    54712
Apple     18922
AMD       15767
Intel     10599
Name: count, dtype: int64

In [246]:
data=pd.get_dummies(data,columns=["gpu_brand"], dtype='int8')

In [247]:
data.columns

Index(['device_type', 'form_factor', 'cpu_model', 'cpu_cores', 'cpu_threads',
       'cpu_boost_ghz', 'gpu_model', 'gpu_tier', 'vram_gb', 'ram_gb',
       ...
       'cpu_base_ghz_3.2', 'cpu_base_ghz_3.4', 'cpu_boost_ghz_3.4',
       'cpu_boost_ghz_3.5', 'cpu_boost_ghz_3.6', 'cpu_boost_ghz_Other',
       'gpu_brand_AMD', 'gpu_brand_Apple', 'gpu_brand_Intel',
       'gpu_brand_NVIDIA'],
      dtype='str', length=106)

In [248]:
data=data.drop(columns="gpu_model")

In [249]:
data["gpu_tier"].value_counts()

gpu_tier
3    23144
2    21256
1    19259
4    19135
5    11844
6     5362
Name: count, dtype: int64

In [250]:
data=pd.get_dummies(data,columns=["gpu_tier"], dtype='int8')

In [251]:
data["vram_gb"].value_counts()

vram_gb
4     20517
6     17930
8     16382
0     13389
10    13251
12     7835
2      7368
16     3328
Name: count, dtype: int64

In [252]:
data=pd.get_dummies(data,columns=["vram_gb"], dtype='int8')

In [253]:
data["ram_gb"].value_counts()

ram_gb
32     22565
16     22258
8      18482
64     16973
96      9172
128     2578
24      2258
40      1965
72      1555
104      817
48       501
80       374
136      249
112      193
144       60
Name: count, dtype: int64

In [254]:
top4 = data["ram_gb"].value_counts().index[:4] 
data['ram_gb_mod'] = data['ram_gb'].apply(lambda x: x if x in top4 else 'Other')
data=pd.get_dummies(data,columns=['ram_gb_mod'], prefix='ram_gb', dtype='int8')


In [255]:
data["storage_type"].value_counts()

storage_type
NVMe      45059
SSD       24937
HDD       15023
Hybrid    14981
Name: count, dtype: int64

In [256]:
data=pd.get_dummies(data,columns=["storage_type"], dtype='int8')

In [257]:
data["storage_gb"].value_counts()

storage_gb
512     35236
1024    30137
256     19816
2048    11839
4096     2972
Name: count, dtype: int64

In [258]:
data=pd.get_dummies(data,columns=["storage_gb"], dtype='int8')

In [259]:
data["storage_drive_count"].value_counts()

storage_drive_count
1    62775
2    25993
3     7191
4     4041
Name: count, dtype: int64

In [260]:
data=pd.get_dummies(data,columns=["storage_drive_count"], dtype='int8')

In [261]:
data["display_type"].value_counts()

display_type
LED         32000
OLED        21910
IPS         17742
Mini-LED    12188
QLED        10069
VA           6091
Name: count, dtype: int64

In [262]:
data=pd.get_dummies(data,columns=["display_type"], dtype='int8')

In [263]:
data["display_size_in"].value_counts()

display_size_in
15.6    22651
14.0    19210
27.0    16790
24.0    11255
32.0     8970
16.0     7200
13.3     7165
17.3     3618
34.0     3141
Name: count, dtype: int64

In [264]:
top4 = data["display_size_in"].value_counts().index[:4] 
data['display_size_in_mod'] = data['display_size_in'].apply(lambda x: x if x in top4 else 'Other')
data=pd.get_dummies(data,columns=['display_size_in_mod'], prefix='display_size_in', dtype='int8')


In [265]:
data.columns

Index(['device_type', 'form_factor', 'cpu_model', 'cpu_cores', 'cpu_threads',
       'cpu_boost_ghz', 'ram_gb', 'display_size_in', 'resolution',
       'refresh_hz',
       ...
       'display_type_LED', 'display_type_Mini-LED', 'display_type_OLED',
       'display_type_QLED', 'display_type_VA', 'display_size_in_14.0',
       'display_size_in_15.6', 'display_size_in_24.0', 'display_size_in_27.0',
       'display_size_in_Other'],
      dtype='str', length=142)

In [266]:
data["resolution"].value_counts()

resolution
1920x1080    47993
2560x1600    14887
2560x1440    14509
3840x2160    10950
2880x1800     5954
3440x1440     5707
Name: count, dtype: int64

In [267]:
data=pd.get_dummies(data,columns=["resolution"], dtype='int8')

In [268]:
data["refresh_hz"].value_counts()

refresh_hz
60     44991
120    22000
90     12078
144    12004
165     6014
240     2913
Name: count, dtype: int64

In [269]:
data=pd.get_dummies(data,columns=["refresh_hz"], dtype='int8')

In [270]:
data["battery_wh"].value_counts()

battery_wh
0     40156
70    13110
60    11983
80    10659
56     9135
90     7196
50     4808
99     2953
Name: count, dtype: int64

In [271]:
data=pd.get_dummies(data,columns=["battery_wh"], dtype='int8')

In [272]:
data["charger_watts"].value_counts()

charger_watts
0      40156
90     16749
65     14892
120    13056
180     6089
45      6019
240     3039
Name: count, dtype: int64

In [273]:
data=pd.get_dummies(data,columns=["charger_watts"], dtype='int8')

In [274]:
data["psu_watts"].value_counts()

psu_watts
0       59844
650      8831
750      7923
550      7179
850      5625
450      4149
1000     2818
300      2447
1200     1184
Name: count, dtype: int64

In [275]:
data=pd.get_dummies(data,columns=["psu_watts"], dtype='int8')

In [276]:
data["wifi"].value_counts()

wifi
Wi-Fi 6     46149
Wi-Fi 6E    25923
Wi-Fi 5     19926
Wi-Fi 7      8002
Name: count, dtype: int64

In [277]:
data=pd.get_dummies(data,columns=["wifi"], dtype='int8')

In [278]:
data["bluetooth"].value_counts()

bluetooth
5.2    28065
5.0    25980
5.1    21951
5.3    18065
4.2     5939
Name: count, dtype: int64

In [279]:
data=pd.get_dummies(data,columns=["bluetooth"], dtype='int8')

In [280]:
data["weight_kg"].value_counts()

weight_kg
1.50     6352
6.00     6319
1.37     5338
8.00     5042
1.70     5026
7.00     4958
1.57     4369
4.00     4043
1.30     4010
9.00     3815
10.00    3658
1.17     3391
2.00     3287
5.00     3215
11.00    2798
1.87     2676
12.00    2184
1.53     1981
1.32     1971
13.00    1733
1.10     1720
1.73     1585
1.52     1571
2.50     1566
0.97     1530
15.00    1360
1.12     1314
1.33     1311
2.37     1305
1.64     1060
16.00    1031
1.82     1015
2.03      988
1.84      740
3.00      690
1.44      656
1.13      622
2.87      601
0.92      586
2.14      498
2.32      484
2.53      475
1.24      288
2.64      279
3.03      238
2.82      224
3.14       97
Name: count, dtype: int64

In [281]:
data=data.drop(columns="weight_kg")

In [282]:
data["warranty_months"].value_counts()

warranty_months
24    40070
12    39967
36    14956
48     5007
Name: count, dtype: int64

In [283]:
data=pd.get_dummies(data,columns=["warranty_months"], dtype='int8')

In [284]:
data

,device_type,form_factor,cpu_model,cpu_cores,cpu_threads,cpu_boost_ghz,ram_gb,display_size_in,price,brand_ASUS,...,wifi_Wi-Fi 7,bluetooth_4.2,bluetooth_5.0,bluetooth_5.1,bluetooth_5.2,bluetooth_5.3,warranty_months_12,warranty_months_24,warranty_months_36,warranty_months_48
0,0,ATX,Intel i5-11129,12,24,3.8,16,27.0,1383.99,0,...,0,0,0,1,0,0,0,0,1,0
1,1,Mainstream,Intel i7-11114,12,24,3.6,64,16.0,2274.99,0,...,0,0,0,0,0,1,1,0,0,0
2,0,SFF,AMD Ryzen 5 5168,8,16,3.6,8,32.0,1879.99,0,...,0,0,1,0,0,0,0,1,0,0
3,0,ATX,AMD Ryzen 5 7550,6,12,3.6,16,27.0,1331.99,0,...,0,0,0,0,1,0,0,0,1,0
4,1,Gaming,AMD Ryzen 7 6230,16,32,3.9,96,15.6,2681.99,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,Mainstream,Intel i7-13721,12,24,3.7,32,14.0,1712.99,1,...,0,0,0,1,0,0,0,1,0,0
99996,1,Ultrabook,AMD Ryzen 5 5117,8,16,3.2,8,14.0,1258.99,0,...,0,0,0,1,0,0,1,0,0,0
99997,1,Mainstream,Intel i5-12677,6,10,3.2,16,14.0,1686.99,1,...,0,1,0,0,0,0,1,0,0,0
99998,1,Mainstream,AMD Ryzen 7 4590,12,24,3.5,32,15.6,2164.99,1,...,0,0,0,0,0,1,0,1,0,0


In [285]:
data.columns[:100]

Index(['device_type', 'form_factor', 'cpu_model', 'cpu_cores', 'cpu_threads',
       'cpu_boost_ghz', 'ram_gb', 'display_size_in', 'price', 'brand_ASUS',
       'brand_Acer', 'brand_Apple', 'brand_Dell', 'brand_Gigabyte', 'brand_HP',
       'brand_Lenovo', 'brand_MSI', 'brand_Razer', 'brand_Samsung',
       'release_year_2018', 'release_year_2019', 'release_year_2020',
       'release_year_2021', 'release_year_2022', 'release_year_2023',
       'release_year_2024', 'release_year_2025', 'os_ChromeOS', 'os_Linux',
       'os_Windows', 'os_macOS', 'form_factor_ATX', 'form_factor_Gaming',
       'form_factor_Mainstream', 'form_factor_Other', 'form_factor_Ultrabook',
       'cpu_brand_AMD', 'cpu_brand_Apple', 'cpu_brand_Intel',
       'cpu_model_Apple M1', 'cpu_model_Apple M1 Max',
       'cpu_model_Apple M1 Pro', 'cpu_model_Apple M2',
       'cpu_model_Apple M2 Max', 'cpu_model_Apple M2 Pro',
       'cpu_model_Apple M3', 'cpu_model_Apple M3 Max',
       'cpu_model_Apple M3 Pro', 'cpu_model

In [286]:
data = data.drop(columns=[
    'form_factor',
    'cpu_model',
    'cpu_cores',
    'cpu_threads',
    'cpu_boost_ghz',
    'ram_gb'
])


In [287]:
data

,device_type,display_size_in,price,brand_ASUS,brand_Acer,brand_Apple,brand_Dell,brand_Gigabyte,brand_HP,brand_Lenovo,...,wifi_Wi-Fi 7,bluetooth_4.2,bluetooth_5.0,bluetooth_5.1,bluetooth_5.2,bluetooth_5.3,warranty_months_12,warranty_months_24,warranty_months_36,warranty_months_48
0,0,27.0,1383.99,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,16.0,2274.99,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0,32.0,1879.99,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
3,0,27.0,1331.99,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,1,15.6,2681.99,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,14.0,1712.99,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
99996,1,14.0,1258.99,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0
99997,1,14.0,1686.99,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
99998,1,15.6,2164.99,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [288]:
data = data.drop(columns=["display_size_in"])

In [289]:
data

,device_type,price,brand_ASUS,brand_Acer,brand_Apple,brand_Dell,brand_Gigabyte,brand_HP,brand_Lenovo,brand_MSI,...,wifi_Wi-Fi 7,bluetooth_4.2,bluetooth_5.0,bluetooth_5.1,bluetooth_5.2,bluetooth_5.3,warranty_months_12,warranty_months_24,warranty_months_36,warranty_months_48
0,0,1383.99,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,2274.99,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0,1879.99,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
3,0,1331.99,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,1,2681.99,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,1712.99,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
99996,1,1258.99,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
99997,1,1686.99,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
99998,1,2164.99,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [290]:
data.columns.value_counts()

device_type           1
price                 1
brand_ASUS            1
brand_Acer            1
brand_Apple           1
                     ..
bluetooth_5.3         1
warranty_months_12    1
warranty_months_24    1
warranty_months_36    1
warranty_months_48    1
Name: count, Length: 175, dtype: int64

In [291]:
x = data.drop("price", axis=1)
y = data["price"]

In [292]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [293]:
x_train.dtypes


device_type           int8
brand_ASUS            int8
brand_Acer            int8
brand_Apple           int8
brand_Dell            int8
                      ... 
bluetooth_5.3         int8
warranty_months_12    int8
warranty_months_24    int8
warranty_months_36    int8
warranty_months_48    int8
Length: 174, dtype: object

In [294]:
x_train = tf.constant(x_train, dtype=tf.float32)
x_test = tf.constant(x_test, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

In [295]:
x_train.shape

TensorShape([80000, 174])

In [296]:
x_test.shape

TensorShape([20000, 174])

In [297]:
model = tf.keras.Sequential([
    Input(shape=(x_train.shape[1],)), 
    Dense(120, activation='relu'),
    Dense(60, activation='relu'),
    Dense(1)  
])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 120)            │        21,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 60)             │         7,260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            61 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,321 (110.63 KB)

 Trainable params: 28,321 (110.63 KB)

 Non-trainable params: 0 (0.00 B)

In [298]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [299]:
reg = model.fit(x_train, y_train, batch_size=40, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 404us/step - loss: 294497.4375 - mean_absolute_error: 293.1683 - val_loss: 48713.3906 - val_mean_absolute_error: 145.3963
Epoch 2/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 353us/step - loss: 49695.9805 - mean_absolute_error: 145.1641 - val_loss: 46782.4336 - val_mean_absolute_error: 141.6393
Epoch 3/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 338us/step - loss: 48785.9219 - mean_absolute_error: 143.0846 - val_loss: 46575.7539 - val_mean_absolute_error: 141.2272
Epoch 4/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 344us/step - loss: 48501.4766 - mean_absolute_error: 142.6575 - val_loss: 46492.6953 - val_mean_absolute_error: 140.9140
Epoch 5/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 338us/step - loss: 48342.3477 - mean_absolute_error: 142.3793 - val_loss: 46586.7969 - val_mean_absolute_error: 140.1280
Epoch 6/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 342us/step - loss: 48245.3828 - mean_absolute_error: 142.2157 - val_loss: 46366.1875 - val_mean_absolute_error: 140.39

In [302]:
pred_train_nn = model.predict(x_train)
pred_test_nn = model.predict(x_test)


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 156us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 155us/step


In [304]:
train_mae = mean_absolute_error(y_train, pred_train_nn)
train_mse = mean_squared_error(y_train, pred_train_nn)
train_r2 = r2_score(y_train, pred_train_nn)

test_mae = mean_absolute_error(y_test, pred_test_nn)
test_mse = mean_squared_error(y_test, pred_test_nn)
test_r2 = r2_score(y_test, pred_test_nn)
print("TRAIN METRICS")
print(f"MAE  : {train_mae:.2f}")
print(f"RMSE : {train_mse:.2f}")
print(f"R²   : {train_r2:.4f}")

print("\nTEST METRICS")
print(f"MAE  : {test_mae:.2f}")
print(f"RMSE : {test_mse:.2f}")
print(f"R²   : {test_r2:.4f}")

TRAIN METRICS
MAE  : 139.86
RMSE : 45086.81
R²   : 0.8666

TEST METRICS
MAE  : 143.54
RMSE : 47566.16
R²   : 0.8571
